# 0. 필요 라이브러리 및 변수, 함수 정의

## import library & package

In [1]:
import os
import numpy as np
import pandas as pd
import torch

from tqdm import tqdm
from itertools import combinations
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoModel, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import load_dataset, load_metric
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

## Define variable data

In [2]:
code_folder = "data/code/"
problem_folders = os.listdir(code_folder)

## Define function

In [3]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preprocess_lines = []
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            preprocess_lines.append(line)
        preprocessed_script = '\n'.join(preprocess_lines) # 개행 문자로 합침
        
    return preprocessed_script

## Preprocessing

In [4]:
preprocess_scripts = []
problem_nums = []

# 300개 Sample code에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder))
    problem_num = problem_folder
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preprocess_scripts.append(preprocessed_script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|█████████████████████████████████████████| 300/300 [00:08<00:00, 33.79it/s]


In [5]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums})
df.head(10)

,code,problem_num
0,"x, k, d = map(int, input().split())\nx = abs(x...",problem092
1,"X,K,D = map(int,input().split())\nX = abs(X)\n...",problem092
2,"X, K, D = map(int,input().split())\nX = abs(X)...",problem092
3,"import sys\nX,K,D= map(int,input().split())\nt...",problem092
4,"X,K,D = map(int,input().split())\nX = abs(X)\n...",problem092
5,"X, K, D=(map(int, input().split()))\nX=abs(X)\...",problem092
6,"X, K, D = map(int, input().split())\nX = abs(X...",problem092
7,"X, K, D = map(int,input().split())\nans = abs(...",problem092
8,"import sys\nX, K, D = map(int, input().split()...",problem092
9,"x, k, d = map(int, input().split())\nx = abs(x...",problem092


In [6]:
len(df)

45101

## Create Data Set

### Tokenizer 수행, microsoft에서 개발한 사전 학습 모델인 graphcodebert 사용

In [7]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

df['tokens'] = df['code'].apply(tokenizer.tokenize) # Sample code를 Tokenization해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가
df.describe()

Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


,len
count,45101.000000
mean,160.123789
std,500.930345
min,5.000000
25%,61.000000
50%,108.000000
75%,200.000000
max,97566.000000


In [8]:
# AutoTokenizer에서 사용하는 모델인 graphcodebert가 512이상 길이의 토큰을 처리하지 못하므로
# 토큰의 길이가 512보다 긴건 Drop함
ndf = df[df['len'] <= 512].reset_index(drop=True)
ndf.describe()

,len
count,43647.000000
mean,137.920842
std,104.933475
min,5.000000
25%,60.000000
50%,104.000000
75%,187.000000
max,512.000000


In [9]:
ndf.head()

,code,problem_num,tokens,len
0,"x, k, d = map(int, input().split())\nx = abs(x...",problem092,"[x, ,, Ġk, ,, Ġd, Ġ=, Ġmap, (, int, ,, Ġinput,...",100
1,"X,K,D = map(int,input().split())\nX = abs(X)\n...",problem092,"[X, ,, K, ,, D, Ġ=, Ġmap, (, int, ,, input, ()...",137
2,"X, K, D = map(int,input().split())\nX = abs(X)...",problem092,"[X, ,, ĠK, ,, ĠD, Ġ=, Ġmap, (, int, ,, input, ...",92
3,"import sys\nX,K,D= map(int,input().split())\nt...",problem092,"[import, Ġsys, Ċ, X, ,, K, ,, D, =, Ġmap, (, i...",234
4,"X,K,D = map(int,input().split())\nX = abs(X)\n...",problem092,"[X, ,, K, ,, D, Ġ=, Ġmap, (, int, ,, input, ()...",117


In [10]:
# train과 validation data set 분리
train_df, valid_df, train_label, valid_label = train_test_split(
        ndf,
        ndf['problem_num'],
        random_state=42,
        test_size=0.1,
        stratify=ndf['problem_num']
    )

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

In [11]:
train_df.head()

,code,problem_num,tokens,len
0,import math\nn=int(input());\narr=[]\nj=2\nwhi...,problem136,"[import, Ġmath, Ċ, n, =, int, (, input, ());, ...",140
1,"x,y,xx,yy=map(float, input().split())\nprint('...",problem029,"[x, ,, y, ,, xx, ,, yy, =, map, (, float, ,, Ġ...",44
2,"import math\nn,k = (int(x) for x in input().sp...",problem099,"[import, Ġmath, Ċ, n, ,, k, Ġ=, Ġ(, int, (, x,...",143
3,"import math\nnum_list = input().split("" "")\npr...",problem086,"[import, Ġmath, Ċ, num, _, list, Ġ=, Ġinput, (...",49
4,n = int(input())\na = 10000\nans = (a - n) % 1...,problem108,"[n, Ġ=, Ġint, (, input, ()), Ċ, a, Ġ=, Ġ10000,...",25


**stratify (classification을 다룰 때 매우 중요한 옵션)**
- default=None
- stratify 값을 target으로 지정해주면 각각의 class 비율을 train / validation에 유지해 줌
    - 한 쪽에 쏠려서 분배되는 것을 방지
- 만약 이 옵션을 지정해 주지 않고 분류 문제를 다룬다면, 성능의 차이가 많이 날 수 있음

#### Create train data

In [37]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

# code를 토큰화하여 저장, train_df에 저장된 모든 코드들에 대한 token들을 리스트 하나에 저장함
tokenized_corpus = [token for token in train_df['tokens']]
# 토큰화된 code에 대해 상관관계를 계산, 현재 무작위로 설정된 code에 대해서 수행하기 때문에
# 상관관계를 계산하는 코드는 같은 문제를 푸는 코드가 아닐 수 있음.
bm25 = BM25Okapi(tokenized_corpus)

In [53]:
total_positive_pairs = []
total_negative_pairs = []

In [54]:
for problem in tqdm(problems): # 문제 번호 차례대로 반복
    # 문제번호가 problem_num (ex: 001, 002...)인 code를 골라 정답 코드로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code']
    # 같은 문제를 푸는 코드들 중 2개를 고르는 조합을 positive_pairs로 설정
    positive_pairs = list(combinations(solution_codes.to_list(),2))[:1000]

    # solution_codes의 index를 list로 변환
    solution_codes_indices = solution_codes.index.to_list()
    negative_pairs = []

    # 첫번째 positive_pairs의 code를 토큰화함.
    first_tokenized_code = tokenizer.tokenize(positive_pairs[0][0])
    # 첫번째 코드와 가장 유사한 code 순서로 negative_code ranking을 설정함.
    negative_code_scores = bm25.get_scores(first_tokenized_code)
    # 내림차순, score가 가장 높은, 즉 유사한 순서대로 ranking을 매김
    negative_code_ranking = negative_code_scores.argsort()[::-1]
    ranking_idx = 0
    
    # 정답 코드에 대해
    for solution_code in solution_codes:
        negative_solutions = []
        # negative_solutions의 길이가 positive_pairs 쌍을 정답 코드 길이만큼 나눈 것보다 작을때
        # 원래 positive_pairs의 길이는 5000~9000, solution_codes의 길이는 100 ~ 120
        # 그럼 negative_solutions의 길이는 대략 50 ~ 90?, 생각보다 negative code가 별로 안들어가고있음
        # positive_pairs의 길이는 1000, solution_codes의 길이는  100 ~ 120
        # 그럼 negative_solutions의 길이는 10?
        while len(negative_solutions) < len(positive_pairs) // len(solution_codes): # negative_code가 더 많이 들어가게 (대략 250개 정도)
            high_score_idx = negative_code_ranking[ranking_idx]
            
            # 유사도가 높은 코드의 index가 동일한 문제에 대한 풀이가 아닐때 negative 코드에 추가
            if high_score_idx not in solution_codes_indices:
                negative_solutions.append(train_df['code'].iloc[high_score_idx])
            # 동일한 문제면 추가하지 않고 다음 ranking 탐색
            ranking_idx += 1
        
        # 정답 코드와 negative 코드를 묶어서 pairs 리스트에 추가
        for negative_solution in negative_solutions:
            negative_pairs.append((solution_code, negative_solution))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|█████████████████████████████████████████| 300/300 [11:33<00:00,  2.31s/it]


In [55]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  300000
total_negative length is  282622


In [56]:
# 쌍으로 묶인 pos, neg 코드를 각각 나눔
pos_code1 = list(map(lambda x:x[0],total_positive_pairs))
pos_code2 = list(map(lambda x:x[1],total_positive_pairs))

neg_code1 = list(map(lambda x:x[0],total_negative_pairs))
neg_code2 = list(map(lambda x:x[1],total_negative_pairs))

# pos와 neg쌍에 대해 라벨링, pos는 모두 동일한 코드이므로 label이 1, neg는 0
pos_label = [1]*len(pos_code1)
neg_label = [0]*len(neg_code1)

# pos_code1과 neg_code1에는 정답 코드가 들어있으므로 합쳐서 total_code1이라는 변수로 선언
pos_code1.extend(neg_code1)
total_code1 = pos_code1
# pos_code2과 neg_code2에는 비교 대상 코드가 들어있으므로 합쳐서 total_code2라는 변수로 선언
pos_code2.extend(neg_code2)
total_code2 = pos_code2
# label 합침
pos_label.extend(neg_label)
total_label = pos_label

# DataFrame으로 선언
pair_data = pd.DataFrame(data={
    'code1':total_code1,
    'code2':total_code2,
    'similar':total_label
})


pair_data = pair_data.sample(frac=1).reset_index(drop=True)

pair_data.to_csv('data/train_data.csv',index=False)

#valid_df에도 동일하게...

#### Create validation data

In [57]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

# code를 토큰화하여 저장, train_df에 저장된 모든 코드들에 대한 token들을 리스트 하나에 저장함
tokenized_corpus = [token for token in valid_df['tokens']]
# 토큰화된 code에 대해 상관관계를 계산, 현재 무작위로 설정된 code에 대해서 수행하기 때문에
# 상관관계를 계산하는 코드는 같은 문제를 푸는 코드가 아닐 수 있음.
bm25 = BM25Okapi(tokenized_corpus)

total_positive_pairs = []
total_negative_pairs = []

In [60]:
for problem in tqdm(problems): # 문제 번호 차례대로 반복
    # 문제번호가 problem_num (ex: 001, 002...)인 code를 골라 정답 코드로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code']
    # 같은 문제를 푸는 코드들 중 2개를 고르는 조합을 positive_pairs로 설정
    positive_pairs = list(combinations(solution_codes.to_list(),2))

    # solution_codes의 index를 list로 변환
    solution_codes_indices = solution_codes.index.to_list()
    negative_pairs = []

    # 첫번째 positive_pairs의 code를 토큰화함.
    first_tokenized_code = tokenizer.tokenize(positive_pairs[0][0])
    # 첫번째 코드와 가장 유사한 code 순서로 negative_code ranking을 설정함.
    negative_code_scores = bm25.get_scores(first_tokenized_code)
    # 내림차순, score가 가장 높은, 즉 유사한 순서대로 ranking을 매김
    negative_code_ranking = negative_code_scores.argsort()[::-1]
    ranking_idx = 0
    
    # 정답 코드에 대해
    for solution_code in solution_codes:
        negative_solutions = []
        # negative_solutions의 길이가 positive_pairs 쌍을 정답 코드 길이만큼 나눈 것보다 작을때
        # 원래 positive_pairs의 길이는 5000~9000, solution_codes의 길이는 100 ~ 120
        # 그럼 negative_solutions의 길이는 대략 50 ~ 90?, 생각보다 negative code가 별로 안들어가고있음
        # positive_pairs의 길이는 1000, solution_codes의 길이는  100 ~ 120
        # 그럼 negative_solutions의 길이는 10?
        while len(negative_solutions) < len(positive_pairs) // len(solution_codes): # negative_code가 더 많이 들어가게 (대략 250개 정도)
            high_score_idx = negative_code_ranking[ranking_idx]
            
            # 유사도가 높은 코드의 index가 동일한 문제에 대한 풀이가 아닐때 negative 코드에 추가
            if high_score_idx not in solution_codes_indices:
                negative_solutions.append(valid_df['code'].iloc[high_score_idx])
            # 동일한 문제면 추가하지 않고 다음 ranking 탐색
            ranking_idx += 1
        
        # 정답 코드와 negative 코드를 묶어서 pairs 리스트에 추가
        for negative_solution in negative_solutions:
            negative_pairs.append((solution_code, negative_solution))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|█████████████████████████████████████████| 300/300 [00:45<00:00,  6.65it/s]


In [61]:
print("total_positive length is ", len(total_positive_pairs))
print("total_negative length is ", len(total_negative_pairs))

total_positive length is  29812
total_negative length is  29577


In [62]:
# 쌍으로 묶인 pos, neg 코드를 각각 나눔
pos_code1 = list(map(lambda x:x[0],total_positive_pairs))
pos_code2 = list(map(lambda x:x[1],total_positive_pairs))

neg_code1 = list(map(lambda x:x[0],total_negative_pairs))
neg_code2 = list(map(lambda x:x[1],total_negative_pairs))

# pos와 neg쌍에 대해 라벨링, pos는 모두 동일한 코드이므로 label이 1, neg는 0
pos_label = [1]*len(pos_code1)
neg_label = [0]*len(neg_code1)

# pos_code1과 neg_code1에는 정답 코드가 들어있으므로 합쳐서 total_code1이라는 변수로 선언
pos_code1.extend(neg_code1)
total_code1 = pos_code1
# pos_code2과 neg_code2에는 비교 대상 코드가 들어있으므로 합쳐서 total_code2라는 변수로 선언
pos_code2.extend(neg_code2)
total_code2 = pos_code2
# label 합침
pos_label.extend(neg_label)
total_label = pos_label

# DataFrame으로 선언
pair_data = pd.DataFrame(data={
    'code1':total_code1,
    'code2':total_code2,
    'similar':total_label
})


pair_data = pair_data.sample(frac=1).reset_index(drop=True)

pair_data.to_csv('data/valid_data.csv',index=False)

#valid_df에도 동일하게...

**BM25: 키워드 기반 랭킹 알고리즘**
- 주어진 쿼리에 대해 문서와의 연관성을 평가하는 랭킹 함수
- Bag-of-words 개념을 사용하여 쿼리에 있는 용어가 각각의 문서에 얼마나 자주 등장하는지를 평가
    - 이때 IDF값을 곱해서 자주 등장하지 않는 단어에 더 큰 가중치를 줌.
    
- 지금까지 한건 Train_data set을 만드는 것.
- 실제 모델에는 위 코드로 만든 train_data와 sample_train을 모두 적용해서 학습시키기
- test는 어떻게?